In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
#задача – собрать ссылки по годам и месяцам, с каждой ссылки собрать авторов, названия статей, ссылки на html файлы, темы, затем с каждой ссылки на html файл собрать текст стаьи

In [ ]:
# генерируем ссылки
import itertools

def generate_urls(category='cs', start_year=2023, end_year=2026, articles_per_page=2000, max_pages_per_month=5):
    base_url = "https://arxiv.org/list/"
    urls = []

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            month_str = f"{month:02d}"
            date_segment = f"{year}-{month_str}"

            for page_num in range(max_pages_per_month):
                skip = page_num * articles_per_page
                url = f"{base_url}{category}/{date_segment}?skip={skip}&show={articles_per_page}"
                urls.append(url)
    return urls

generated_urls = generate_urls(
    category='cs', 
    start_year=2023, 
    end_year=2026, 
    articles_per_page=2000,
    max_pages_per_month=5
)

print("Первые 10 URL:")
for i, url in enumerate(generated_urls[:10]):
    print(url)

print("\nПоследние 10 URL:")
for i, url in enumerate(generated_urls[-10:]):
    print(url)

print(f"\nВсего сгенерировано URL: {len(generated_urls)}")

In [ ]:
import re

In [ ]:
#функция для извлечения текста статей

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
def extract_full_text(html_url):
    if not html_url:
        return "N/A"

    try:
        time.sleep(1) 
        
        response = requests.get(html_url, headers=HEADERS, timeout=20)
        response.raise_for_status()
        
        soup_html = BeautifulSoup(response.text, 'html.parser')
        
        content_div = soup_html.find('div', {'class':"ltx_page_content"})
        
        if content_div:
            full_text = content_div.get_text(separator=' ', strip=True)
            return full_text
        else:
            return "N/A - Content div not found"
            
    except requests.exceptions.RequestException as e:
        return f"N/A - Error loading HTML page: {e}"
    except Exception as e:
        return f"N/A - General error extracting HTML text: {e}"

In [ ]:
from urllib.parse import urljoin

In [ ]:
# функция для извлечения всей необходимой информации по статьям с каждой страницы

def extract_all_article_details(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=20)
    except Exception as e:
        print(f"Ошибка при загрузке списка: {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    articles_list = soup.find('dl', id='articles')
    if not articles_list:
        return []

    dts = articles_list.find_all('dt')
    dds = articles_list.find_all('dd')
    
    all_articles_data = []

    for dt, dd in zip(dts, dds):

        html_link_tag = dt.find('a', title='View HTML')
        
        html_url = None
        full_text = "N/A - No HTML version" 

        if html_link_tag:
            html_url = urljoin("https://arxiv.org", html_link_tag['href'])
            
            print(f"Парсим текст для статьи: {html_url}")
            full_text = extract_full_text(html_url)
        else:
            print("HTML версия отсутствует, пропускаем...")

        t_div = dd.find('div', class_='list-title')
        title = t_div.get_text(strip=True).replace('Title:', '') if t_div else "N/A"
        a_div = dd.find('div', class_='list-authors')
        authors = a_div.get_text(strip=True).replace('Authors:', '') if a_div else "N/A"
        s_div = dd.find('div', class_='list-subjects')
        subjects = s_div.get_text(strip=True).replace('Subjects:', '') if s_div else "N/A"

        all_articles_data.append({
            'title': title,
            'authors': authors,
            'subjects': subjects,
            'html_url': html_url,
            'full_text': full_text
        })
    
    return all_articles_data

In [ ]:
#собираем информацию со всех сгенирированных ссылок и создаем data frame

In [ ]:
import os

In [ ]:
FILE = 'arxiv_parsed_data.csv'

In [ ]:
def main_parser(generated_urls):
    all_data = []
    
    # Загружаем уже существующие данные, чтобы продолжить с места остановки
    processed_urls = set()
    if os.path.exists(FILE):
        existing_df = pd.read_csv(FILE)
        if 'source_page_url' in existing_df.columns:
            processed_urls = set(existing_df['source_page_url'].unique())
            all_data = existing_df.to_dict('records')
            print(f"Найдено сохраненных данных. Пропущено страниц: {len(processed_urls)}")

    # Основной цикл по списку сгенерированных ссылок
    for url in generated_urls:
        if url in processed_urls:
            continue
            
        print(f"Начинаю обработку страницы: {url}")
        
        try:
            articles = extract_all_article_details(url)
            
            if articles:
                for a in articles:
                    a['source_page_url'] = url
                
                all_data.extend(articles)
                
                df_temp = pd.DataFrame(all_data)
                df_temp.to_csv(FILE, index=False, encoding='utf-8')
                
                print(f"Успешно: собрано {len(articles)} статей. Данные обновлены в {FILE}")
            
            processed_urls.add(url)
            
            time.sleep(2)

        except Exception as e:
            print(f"КРИТИЧЕСКАЯ ОШИБКА на странице {url}: {e}")
            print("Все собранные данные сохранены. Можно запустить код снова позже.")
            if all_data:
                pd.DataFrame(all_data).to_csv(FILE, index=False, encoding='utf-8')
            continue 

    return pd.DataFrame(all_data)

In [ ]:
arxiv_df = main_parser(generated_urls)